In [1]:
include("src/NursesScheduling.jl")
using .NurseSchedules
using Logging

logger = ConsoleLogger(stderr, Logging.Debug)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x00000032) open, 0 bytes waiting))), Debug, Logging.default_metafmt, true, 0, Dict{Any,Int64}())

In [2]:
nurse_schedule = Schedule("./schedules/schedule_2016_august.json")

┌ Info: Schedule loaded!
└ @ Main.NurseSchedules /home/prenc/Documents/nurse/nurse_scheduling_problem/src/Schedule.jl:13


Schedule(Dict{String,Any}("shifts" => Dict{String,Any}("nurse_9" => Any["U", "U", "U", "U", "U", "W", "DN", "W", "DN", "W"  …  "DN", "W", "DN", "N", "W", "DN", "U", "U", "U", "U"],"nurse_2" => Any["W", "D", "R", "N", "W", "D", "W", "N", "W", "N"  …  "W", "DN", "PN", "W", "DN", "W", "W", "D", "R", "N"],"nurse_3" => Any["U", "U", "U", "U", "W", "DN", "W", "DN", "W", "DN"  …  "W", "W", "W", "D", "N", "W", "U", "U", "U", "U"],"nurse_6" => Any["U", "U", "U", "U", "U", "U", "U", "U", "U", "U"  …  "W", "D", "DN", "N", "W", "DN", "U", "U", "U", "U"],"nurse_8" => Any["D", "N", "N", "W", "D", "W", "W", "D", "N", "W"  …  "R", "W", "W", "D", "W", "W", "D", "N", "N", "W"],"nurse_4" => Any["W", "W", "D", "D", "N", "W", "D", "N", "W", "D"  …  "U", "U", "U", "U", "W", "W", "W", "W", "D", "D"],"nurse_5" => Any["W", "W", "W", "W", "N", "W", "W", "W", "W", "W"  …  "DN", "N", "R", "W", "PN", "W", "W", "W", "W", "W"],"nurse_7" => Any["N", "W", "N", "W", "U", "U", "U", "U", "U", "U"  …  "W", "N", "W", "N", 

In [3]:
schedule_shifts = get_shifts(nurse_schedule)
workers, shifts = schedule_shifts
month_info = get_month_info(nurse_schedule)
workers_info = get_workers_info(nurse_schedule)

Dict{String,Any} with 1 entry:
  "time" => Dict{String,Any}("nurse_9"=>"FULL","nurse_2"=>"FULL","nurse_3"=>"FU…

In [4]:
with_logger(logger) do
    get_max_nbhd_size(nurse_schedule)
end

┌ Debug: Neighbors number from addition: 768
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:14
┌ Debug: Neighbors number from deletion: 118
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:16
┌ Debug: Neighbors number from swap: 124
└ @ Main.NurseSchedules.Neighborhood_gen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:29


1010

In [5]:
nbhd = Neighborhood(shifts)
length(neighborhood)

LoadError: UndefVarError: neighborhood not defined

In [6]:
with_logger(logger) do
    global schedule_score = score(schedule_shifts, month_info, workers_info)
end

┌ Debug: Lacking nurses on day '1'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '2'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '3'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '2' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '4'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '5'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '2' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/s

┌ Debug: Worker 'nurse_9' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:103
┌ Debug: Worker 'nurse_9' does not have a long break in week: '2'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:103
┌ Debug: Worker 'nurse_9' does not have a long break in week: '3'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:103
┌ Debug: Worker 'nurse_9' does not have a long break in week: '4'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:103
┌ Debug: Worker 'nurse_9' does not have a long break in week: '5'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:103
┌ Debug: Worker 'nurse_2' has a disallowed shift sequence in day '27': DN -> PN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ D

3050

In [7]:
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["U" "U" … "U" "U"; "W" "D" … "R" "N"; … ; "N" "W" … "N" "W"; "U" "U" … "U" "U"], 3050)

In [8]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    nbhd = Neighborhood(iter_best[1])
    nghd_scores = map(shifts -> score((workers, shifts), month_info, workers_info), nbhd)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    iter_best = (nbhd[iter_best_idx], nghd_scores[iter_best_idx])
    
    no_improvement_iters += 1
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global no_improvement_iters = 0
        global best_shifts = iter_best
    end
    if no_improvement_iters > 20
        println("Local minimum! Stopping in iteration '$(i)'")
        break
    end
end

Improved from 3050 to 3000 in iteration '1'
Improved from 3000 to 2950 in iteration '2'
Improved from 2950 to 2900 in iteration '3'
Improved from 2900 to 2851 in iteration '4'
Improved from 2851 to 2802 in iteration '5'
Improved from 2802 to 2753 in iteration '6'
Improved from 2753 to 2713 in iteration '7'
Improved from 2713 to 2673 in iteration '8'
Improved from 2673 to 2633 in iteration '9'
Improved from 2633 to 2593 in iteration '10'
Improved from 2593 to 2553 in iteration '11'
Improved from 2553 to 2513 in iteration '12'
Improved from 2513 to 2473 in iteration '13'
Improved from 2473 to 2433 in iteration '14'
Improved from 2433 to 2393 in iteration '15'
Improved from 2393 to 2353 in iteration '16'
Improved from 2353 to 2313 in iteration '17'
Improved from 2313 to 2273 in iteration '18'
Improved from 2273 to 2233 in iteration '19'
Improved from 2233 to 2193 in iteration '20'
Improved from 2193 to 2153 in iteration '21'
Improved from 2153 to 2114 in iteration '22'
Improved from 2114 

In [9]:
with_logger(logger) do
    score((workers, best_shifts[1]), month_info, workers_info)
end

┌ Debug: Lacking nurses on day '3'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '5'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Lacking nurses on day '34'.
│ Expected '4', got '2' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:60
┌ Debug: Total lack of nurses penalty: 80
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:64
┌ Debug: Worker 'nurse_9' has a disallowed shift sequence in day '6': N -> DN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_9' has a disallowed shift sequence in day '7': DN -> PN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ 

┌ Debug: Worker 'nurse_8' has a disallowed shift sequence in day '13': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_8' has a disallowed shift sequence in day '16': DN -> N
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_8' has a disallowed shift sequence in day '24': N -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_8' has a disallowed shift sequence in day '34': N -> DN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '1': DN -> DN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:85
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '2': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Doc

641